In [2]:
%matplotlib inline

from astropy.io import fits

In [3]:
hdu_list = fits.open('/Users/tdavid/Dropbox (Simons Foundation)/gaia/gaia_edr3_100pc_10sigma_ruwe_lt_2.fits')


In [4]:
print(hdu_list[1].columns)


ColDefs(
    name = 'SOURCE_ID'; format = 'K'
    name = 'RA'; format = 'D'
    name = 'DEC'; format = 'D'
    name = 'PMRA'; format = 'D'
    name = 'PMDEC'; format = 'D'
    name = 'PARALLAX'; format = 'D'
    name = 'PHOT_G_MEAN_MAG'; format = 'E'
    name = 'PHOT_BP_MEAN_MAG'; format = 'E'
    name = 'PHOT_RP_MEAN_MAG'; format = 'E'
    name = 'PARALLAX_ERROR'; format = 'E'
    name = 'PMRA_ERROR'; format = 'E'
    name = 'PMDEC_ERROR'; format = 'E'
    name = 'RUWE'; format = 'E'
    name = 'DR2_RADIAL_VELOCITY'; format = 'E'
    name = 'DR2_RADIAL_VELOCITY_ERROR'; format = 'E'
)


In [5]:
from astropy.table import Table

data = Table(hdu_list[1].data)

In [6]:
data

SOURCE_ID,RA,DEC,PMRA,PMDEC,PARALLAX,PHOT_G_MEAN_MAG,PHOT_BP_MEAN_MAG,PHOT_RP_MEAN_MAG,PARALLAX_ERROR,PMRA_ERROR,PMDEC_ERROR,RUWE,DR2_RADIAL_VELOCITY,DR2_RADIAL_VELOCITY_ERROR
int64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32
2060772110911836800,302.3525977645142,37.165753725512126,8.680704165848157,34.670837848110416,11.705682067194678,12.703084,13.599332,11.721476,0.021492396,0.025890637,0.025285045,1.9566131,-7.85652,7.9898043
2059258495721163392,302.13302571238563,36.792492105162175,36.05864729447675,42.91674356089226,16.185215170124504,10.114978,10.611381,9.462236,0.01255183,0.012464459,0.013209747,0.945186,-54.715717,0.19426277
2061572898979115136,302.80354182430113,37.96335974983874,144.72030573921424,-119.28244066810987,29.311371469120427,10.984141,12.066932,9.946714,0.02468575,0.028205652,0.026676489,0.9955967,-66.92955,0.3772401
2061568157329376512,302.790321535523,37.82426655968013,-24.204767210466066,-12.432913305494269,40.80143772910702,20.1479,20.656479,18.52732,2.6022437,1.2262226,1.1425165,1.4932408,nan,nan
2062090494058090624,303.66528092717556,38.83871418646031,-209.5451825272002,-250.2190668124997,21.652157507119146,15.339985,17.07953,14.075593,0.02630454,0.028537313,0.035668463,1.1544235,nan,nan
2062086645766639360,303.795505608646,38.79240687863123,37.57267489360596,-3.6004085621494992,11.93502153087604,13.076068,14.104916,12.067883,0.010857461,0.010486168,0.012651689,0.98765403,-2.039972,1.3297917
2060917830543284864,303.3750578820874,38.07283112638199,-22.943743994503137,-5.965564994342483,11.80582129536751,15.636205,16.6578,13.851152,0.05422925,0.059133258,0.065529376,1.386878,nan,nan
2060917830555293824,303.3752770465698,38.07302657867787,-26.061923918464068,-4.854252725076821,11.999595577778209,15.935367,nan,nan,0.06573263,0.09422843,0.095924206,1.1356152,nan,nan
2060715138141213824,303.40582663546536,36.90440920783438,98.88893208153631,29.31819500151483,13.226603390529101,12.359637,13.265699,11.422185,0.009869111,0.009775334,0.010838211,0.99010724,-22.042099,0.30497962


In [8]:
from math import cos, sin
from astropy.coordinates import SkyCoord
import numpy as np

# ===================================================
def uvw(ra, dec, d, pmra, pmde, rv):
    """
    Function to calculate UVW given RA, Dec, Distance, RV, and PMs
    Adapted from http://idlastro.gsfc.nasa.gov/ftp/pro/astro/gal_uvw.pro
    :param ra: Right Ascension in degrees
    :param dec: Declination in degrees
    :param d: Distance in parsecs
    :param pmra: Proper motion in RA in milli-arcseconds/year
    :param pmde: Proper motion in Dec in milli-arcseconds/year
    :param rv: Radial velocity in km/s
    :return: U, V, W in km/s
    """
    k = 4.74047  # Equivalent of 1 A.U/yr in km/s
    A00 = 0.0548755604
    A01 = 0.8734370902
    A02 = 0.4838350155
    A10 = 0.4941094279
    A11 = -0.4448296300
    A12 = 0.7469822445
    A20 = -0.8676661490
    A21 = -0.1980763734
    A22 = 0.4559837762

    # Set as arrays in case ra, dec, etc were lists
    ra = np.array(ra)
    dec = np.array(dec)
    d = np.array(d)
    rv = np.array(rv)
    pmra = np.array(pmra)
    pmde = np.array(pmde)

    radcon = 3.1415926/180  # radian conversion factor

    try:
        cosd = cos(dec * radcon)
        sind = sin(dec * radcon)
        cosa = cos(ra * radcon)
        sina = sin(ra * radcon)
    except TypeError:  # For arrays
        cosd = np.array(map(cos, dec * radcon))
        sind = np.array(map(sin, dec * radcon))
        cosa = np.array(map(cos, ra * radcon))
        sina = np.array(map(sin, ra * radcon))

    vec1 = rv
    plx = 1000./d
    vec2 = k * pmra/plx
    vec3 = k * pmde/plx

    u = (A00*cosa*cosd + A01*sina*cosd + A02*sind) * vec1 + \
        (-A00*sina + A01*cosa) * vec2 + \
        (-A00*cosa*sind - A01*sina*sind + A02*cosd) * vec3
    v = (A10*cosa*cosd + A11*sina*cosd + A12*sind) * vec1 + \
        (-A10*sina + A11*cosa) * vec2 + \
        (-A10*cosa*sind - A11*sina*sind + A12*cosd) * vec3
    w = (A20*cosa*cosd + A21*sina*cosd + A22*sind) * vec1 + \
        (-A20*sina + A21*cosa) * vec2 + \
        (-A20*cosa*sind - A21*sina*sind + A22*cosd) * vec3
    u = -u  # Flipping U to be positive towards Galactic center

    return u, v, w


# ===================================================
def xyz(ra, dec, d):
    """
    Function to calculate XYZ given RA, Dec, and Distance
    :param ra: Right Ascension in degrees
    :param dec: Declination in degrees
    :param d: Distance in parsecs
    :return: X, Y, Z in parsecs
    """

    ra = np.array(ra)
    dec = np.array(dec)
    d = np.array(d)

    c = SkyCoord(ra=ra, dec=dec, frame='icrs', unit='deg')
    l, b = c.galactic.l.radian, c.galactic.b.radian

    try:
        xgc = d * cos(b) * cos(l)
        ygc = d * cos(b) * sin(l)
        zgc = d * sin(b)
    except TypeError:  # For arrays
        xgc = d * map(cos, b) * map(cos, l)
        ygc = d * map(cos, b) * map(sin, l)
        zgc = d * map(sin, b)

    return xgc, ygc, zgc

# ===================================================

In [9]:
data['DIST'] = 1.0e3/data['PARALLAX']

In [17]:
data['U'],data['V'],data['W'] = np.array(uvw(data['RA']), np.array(data['DEC']), np.array(data['DIST']), np.array(data['PMRA']), np.array(data['PMDEC']), np.array(data['DR2_RADIAL_VELOCITY'])

SyntaxError: unexpected EOF while parsing (<ipython-input-17-b8a45ceeb0cd>, line 1)